In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

# Parameters
learning_rate = 0.01
batch_size = 128
training_epochs = 30

n_classes = 10

In [3]:
layer_width = {
    'layer_1':32,
    'layer_2':64,
    'layer_3':128,
    'fully_connected':512
}

weights = {
    'layer_1':tf.Variable(tf.truncated_normal(
            [5, 5, 1, layer_width['layer_1']])),
    'layer_2':tf.Variable(tf.truncated_normal(
            [5, 5, layer_width['layer_1'], layer_width['layer_2']])),
    'layer_3':tf.Variable(tf.truncated_normal(
            [5, 5, layer_width['layer_2'], layer_width['layer_3']])),
    'fully_connected':tf.Variable(tf.truncated_normal(
        [4*4*128, layer_width['fully_connected']])),
    'out':tf.Variable(tf.truncated_normal(
        [layer_width['fully_connected'], n_classes]))
}

biases = {
    'layer_1':tf.Variable(tf.zeros(layer_width['layer_1'])),
    'layer_2':tf.Variable(tf.zeros(layer_width['layer_2'])),
    'layer_3':tf.Variable(tf.zeros(layer_width['layer_3'])),
    'fully_connected':tf.Variable(tf.zeros(layer_width['fully_connected'])),
    'out':tf.Variable(tf.zeros(n_classes))
}

In [4]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1,strides,strides,1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1,k,k,1],
        strides=[1,k,k,1],
        padding='SAME')

In [7]:
def conv_net(x, weights, biases):
    # Layer 1 - 28x28x1 to 14x14x32
    conv1 = conv2d(x, weights['layer_1'], biases['layer_1'])
    conv1 = maxpool2d(conv1)
    
    # Layer 2 - 14x14x32 to 7x7x64
    conv2 = conv2d(conv1, weights['layer_2'], biases['layer_2'])
    conv2 = maxpool2d(conv2)
    
    # Layer 3 - 7x7x64 to 4x4x128
    conv3 = conv2d(conv2, weights['layer_3'], biases['layer_3'])
    conv3 = maxpool2d(conv3)
    
    # Fully connected layer - 4x4x128 to 512
    # Reshape conv3 output to fit fully connected layer input
    fc1 = tf.reshape(
        conv3,
        [-1, weights['fully_connected'].get_shape().as_list()[0]])
    fc1 = tf.add(
        tf.matmul(fc1, weights['fully_connected']),
        biases['fully_connected'])
    fc1 = tf.nn.tanh(fc1)
    
    # Output layer - class prediction - 512 to 10
    out = tf.add(
        tf.matmul(fc1, weights['out']),
        biases['out'])
    return out

In [9]:
# Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])

logits = conv_net(x, weights, biases)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
            .minimize(loss)

In [14]:
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            sess.run(optimizer, feed_dict={x:batch_x, y:batch_y})
        
        c = sess.run(loss, feed_dict={x:batch_x, y:batch_y})
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c))
    
    print("Optimization finished!")
    
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy: ",
         accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

Epoch: 0001 cost= 5.954862118
Epoch: 0002 cost= 7.171306610
Epoch: 0003 cost= 5.587461948
Epoch: 0004 cost= 6.547703266
Epoch: 0005 cost= 2.620368004
Epoch: 0006 cost= 2.545758963
Epoch: 0007 cost= 2.731041908
Epoch: 0008 cost= 4.273614407
Epoch: 0009 cost= 3.057936668
Epoch: 0010 cost= 2.537177086
Epoch: 0011 cost= 3.236813545
Epoch: 0012 cost= 2.444550991
Epoch: 0013 cost= 2.635271788
Epoch: 0014 cost= 2.328088284
Epoch: 0015 cost= 2.279029131
Epoch: 0016 cost= 2.281298399
Epoch: 0017 cost= 2.295808554
Epoch: 0018 cost= 2.336714983
Epoch: 0019 cost= 2.310711384
Epoch: 0020 cost= 2.307648897
Epoch: 0021 cost= 2.273983240
Epoch: 0022 cost= 2.268383503
Epoch: 0023 cost= 2.272824526
Epoch: 0024 cost= 2.298228979
Epoch: 0025 cost= 2.250019073
Epoch: 0026 cost= 2.279612064
Epoch: 0027 cost= 2.302534819
Epoch: 0028 cost= 2.299911022
Epoch: 0029 cost= 2.323016882
Epoch: 0030 cost= 2.273174047
Optimization finished!
Accuracy:  0.0978
